<a href="https://colab.research.google.com/github/dev26git/NLP_Assignments/blob/main/NLP_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Assignment 1
### Devansh Singh
### 21070126024
### AIML-A2

1) https://www.kaggle.com/datasets/mateibejan/multilingual-lyrics-for-genre-classification?select=train.csv (For A2 Select only english songs from genre Rock, Jazz, HipHop, Metal, Country.From Song Lyrics predict Genre.)

2) DO FOLLOWING Preprocessing STEPS using NLTK or SpaCy:
    a) Remove empty rows. Remove duplicates.
    b) Tokenization, Lemmatization
    b) Data Cleansing: Remove stopwords, remove symbols,remove emojis,     remove URLS, remove http tags, remove excess whitespaces.
    c) Lower the strings, replace abbreviations, fix contractions.

3) Use following as range of values for input variables:
    For 1st set of results use:
    a) batch_size = 4
    b) max_sequence_length = 50
    c) embedding_dim = 50
    d) max_words = 10000
    e) lstm_units = 32
     
    For 2st set of results use:
    a) batch_size = 8
    b) max_sequence_length = 30
    c) embedding_dim = 30
    d) max_words = 25000
    e) lstm_units = 32

4) Use following LSTM:
    a) Single Layer of LSTM for 1st set of results
    b) Two Layers of LSTM for 2nd set of results

5) Compare the results using Classification Report.
Assignment will be assessed as per Rubric and results will be compared with others. Make sure you get the best results.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download('all', quiet=True)

True

In [ ]:
#Importing the Libraries for Prediction

import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv("/content/drive/MyDrive/NLP Assignments/Datasets/NLP_Dataset_1.csv")
df.head()

,Artist,Song,Genre,Language,Lyrics
0,12 stones,world so cold,Rock,en,"It starts with pain, followed by hate\nFueled ..."
1,12 stones,broken,Rock,en,Freedom!\nAlone again again alone\nPatiently w...
2,12 stones,3 leaf loser,Rock,en,"Biting the hand that feeds you, lying to the v..."
3,12 stones,anthem for the underdog,Rock,en,You say you know just who I am\nBut you can't ...
4,12 stones,adrenaline,Rock,en,My heart is beating faster can't control these...


In [ ]:
# keep only english songs

df = df[df['Language'] == 'en']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250197 entries, 0 to 290182
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Artist    250197 non-null  object
 1   Song      250196 non-null  object
 2   Genre     250197 non-null  object
 3   Language  250197 non-null  object
 4   Lyrics    250197 non-null  object
dtypes: object(5)
memory usage: 11.5+ MB


In [ ]:
df.isna().sum()

Artist      0
Song        1
Genre       0
Language    0
Lyrics      0
dtype: int64

In [ ]:
# Drop rows with missing values
df.dropna(inplace=True)

df.isna().sum()

Artist      0
Song        0
Genre       0
Language    0
Lyrics      0
dtype: int64

In [ ]:
# Remove duplicate rows based on all columns
df.drop_duplicates(inplace=True)

In [ ]:
# Drop uneccessary columns

df.drop(columns=['Artist', 'Song', 'Language'], inplace = True)

In [ ]:
# Drop few Genres

keep_genres = ['Rock', 'Jazz', 'Hip-Hop', 'Metal', 'Country']
df = df.loc[df['Genre'].isin(keep_genres)]

In [ ]:
df.Genre.value_counts()

Rock       100478
Metal       17976
Jazz        13313
Hip-Hop      2238
Country      1890
Name: Genre, dtype: int64

In [ ]:
# Sample Lyrics

df.iloc[0,1]

"It starts with pain, followed by hate\nFueled by the endless questions no one can answer\nA stain covers your heart and tears you apart\nJust like a sleeping cancer\nI don't believe men are born to be killers\nI don't believe the world can be saved\nHow did you get here and when did it start?\nAn innocent child with a thorn in his heart\nWhat kind of world do we live in?\nWhere love is divided by hate\nLoosing control of our feelings\nWe all must be dreaming this life away\nIn a world so cold\nAre you sane, where's the shame?\nA moment of time passes by you cannot rewind\nWho's to blame and where did it start?\nIs there a cure for your sickness\nHave you no heart?\nI don't believe men are born to be killers\nI don't believe the world can't be saved\nHow did you get here and when did it start?\nAn innocent child with a thorn in his heart\nWhat kind of world do we live in?\nWhere love is divided by hate\nSelling our soul for no reason\nWe all must be dreaming this life away\nIn a world 

# Preprocessing

In [ ]:
# Remove new lines

df['Lyrics'] = df['Lyrics'].apply(lambda txt: re.sub(r'\n', ' ', txt))

# Sample
df.iloc[0,1]

"It starts with pain, followed by hate Fueled by the endless questions no one can answer A stain covers your heart and tears you apart Just like a sleeping cancer I don't believe men are born to be killers I don't believe the world can be saved How did you get here and when did it start? An innocent child with a thorn in his heart What kind of world do we live in? Where love is divided by hate Loosing control of our feelings We all must be dreaming this life away In a world so cold Are you sane, where's the shame? A moment of time passes by you cannot rewind Who's to blame and where did it start? Is there a cure for your sickness Have you no heart? I don't believe men are born to be killers I don't believe the world can't be saved How did you get here and when did it start? An innocent child with a thorn in his heart What kind of world do we live in? Where love is divided by hate Selling our soul for no reason We all must be dreaming this life away In a world so cold, In a world so col

In [ ]:
# lower

df['Lyrics'] = df['Lyrics'].str.lower()

# Sample
df.iloc[0,1]

"it starts with pain, followed by hate fueled by the endless questions no one can answer a stain covers your heart and tears you apart just like a sleeping cancer i don't believe men are born to be killers i don't believe the world can be saved how did you get here and when did it start? an innocent child with a thorn in his heart what kind of world do we live in? where love is divided by hate loosing control of our feelings we all must be dreaming this life away in a world so cold are you sane, where's the shame? a moment of time passes by you cannot rewind who's to blame and where did it start? is there a cure for your sickness have you no heart? i don't believe men are born to be killers i don't believe the world can't be saved how did you get here and when did it start? an innocent child with a thorn in his heart what kind of world do we live in? where love is divided by hate selling our soul for no reason we all must be dreaming this life away in a world so cold, in a world so col

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.3 MB/s eta 0:00:00


In [ ]:
# Fix contractions
import contractions

df['Lyrics'] = df['Lyrics'].apply(lambda text: contractions.fix(text))

# Sample
df.iloc[0,1]

'it starts with pain, followed by hate fueled by the endless questions no one can answer a stain covers your heart and tears you apart just like a sleeping cancer i do not believe men are born to be killers i do not believe the world can be saved how did you get here and when did it start? an innocent child with a thorn in his heart what kind of world do we live in? where love is divided by hate loosing control of our feelings we all must be dreaming this life away in a world so cold are you sane, where is the shame? a moment of time passes by you cannot rewind who is to blame and where did it start? is there a cure for your sickness have you no heart? i do not believe men are born to be killers i do not believe the world cannot be saved how did you get here and when did it start? an innocent child with a thorn in his heart what kind of world do we live in? where love is divided by hate selling our soul for no reason we all must be dreaming this life away in a world so cold, in a world

In [ ]:
# Remove Punctuations

import string

def remove_punctuations(text):
    return "".join([c for c in text if c not in string.punctuation])

df['Lyrics'] = df['Lyrics'].apply(lambda x: remove_punctuations(x))

# Sample
df.iloc[0,1]

'it starts with pain followed by hate fueled by the endless questions no one can answer a stain covers your heart and tears you apart just like a sleeping cancer i do not believe men are born to be killers i do not believe the world can be saved how did you get here and when did it start an innocent child with a thorn in his heart what kind of world do we live in where love is divided by hate loosing control of our feelings we all must be dreaming this life away in a world so cold are you sane where is the shame a moment of time passes by you cannot rewind who is to blame and where did it start is there a cure for your sickness have you no heart i do not believe men are born to be killers i do not believe the world cannot be saved how did you get here and when did it start an innocent child with a thorn in his heart what kind of world do we live in where love is divided by hate selling our soul for no reason we all must be dreaming this life away in a world so cold in a world so cold t

In [ ]:
nltk.download('all', quiet=True)

True

In [ ]:
from nltk.tokenize import word_tokenize

df['Lyrics'] = df['Lyrics'].apply(lambda text: word_tokenize(text))

# Word tokenize also takes care of extra spaces around the words

# Sample
df.iloc[0,1]

['it',
 'starts',
 'with',
 'pain',
 'followed',
 'by',
 'hate',
 'fueled',
 'by',
 'the',
 'endless',
 'questions',
 'no',
 'one',
 'can',
 'answer',
 'a',
 'stain',
 'covers',
 'your',
 'heart',
 'and',
 'tears',
 'you',
 'apart',
 'just',
 'like',
 'a',
 'sleeping',
 'cancer',
 'i',
 'do',
 'not',
 'believe',
 'men',
 'are',
 'born',
 'to',
 'be',
 'killers',
 'i',
 'do',
 'not',
 'believe',
 'the',
 'world',
 'can',
 'be',
 'saved',
 'how',
 'did',
 'you',
 'get',
 'here',
 'and',
 'when',
 'did',
 'it',
 'start',
 'an',
 'innocent',
 'child',
 'with',
 'a',
 'thorn',
 'in',
 'his',
 'heart',
 'what',
 'kind',
 'of',
 'world',
 'do',
 'we',
 'live',
 'in',
 'where',
 'love',
 'is',
 'divided',
 'by',
 'hate',
 'loosing',
 'control',
 'of',
 'our',
 'feelings',
 'we',
 'all',
 'must',
 'be',
 'dreaming',
 'this',
 'life',
 'away',
 'in',
 'a',
 'world',
 'so',
 'cold',
 'are',
 'you',
 'sane',
 'where',
 'is',
 'the',
 'shame',
 'a',
 'moment',
 'of',
 'time',
 'passes',
 'by',
 'yo

In [ ]:
# Lemmatization
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def get_lemmatized_list(text_list):
    return [lemmatizer.lemmatize(word) for word in text_list]


df['Lyrics'] = df['Lyrics'].apply(lambda text_list: get_lemmatized_list(text_list))

# Sample
df.iloc[0,1]

['it',
 'start',
 'with',
 'pain',
 'followed',
 'by',
 'hate',
 'fueled',
 'by',
 'the',
 'endless',
 'question',
 'no',
 'one',
 'can',
 'answer',
 'a',
 'stain',
 'cover',
 'your',
 'heart',
 'and',
 'tear',
 'you',
 'apart',
 'just',
 'like',
 'a',
 'sleeping',
 'cancer',
 'i',
 'do',
 'not',
 'believe',
 'men',
 'are',
 'born',
 'to',
 'be',
 'killer',
 'i',
 'do',
 'not',
 'believe',
 'the',
 'world',
 'can',
 'be',
 'saved',
 'how',
 'did',
 'you',
 'get',
 'here',
 'and',
 'when',
 'did',
 'it',
 'start',
 'an',
 'innocent',
 'child',
 'with',
 'a',
 'thorn',
 'in',
 'his',
 'heart',
 'what',
 'kind',
 'of',
 'world',
 'do',
 'we',
 'live',
 'in',
 'where',
 'love',
 'is',
 'divided',
 'by',
 'hate',
 'loosing',
 'control',
 'of',
 'our',
 'feeling',
 'we',
 'all',
 'must',
 'be',
 'dreaming',
 'this',
 'life',
 'away',
 'in',
 'a',
 'world',
 'so',
 'cold',
 'are',
 'you',
 'sane',
 'where',
 'is',
 'the',
 'shame',
 'a',
 'moment',
 'of',
 'time',
 'pass',
 'by',
 'you',
 'ca

In [ ]:
# Remove Stopwords
from nltk.corpus import stopwords

stopwords_set = set(stopwords.words('english'))

def remove_stopwords(text_list):
    return [word for word in text_list if word not in stopwords_set]

df['Lyrics'] = df['Lyrics'].apply(lambda text_list: remove_stopwords(text_list))

# Sample
df.iloc[0,1]

['start',
 'pain',
 'followed',
 'hate',
 'fueled',
 'endless',
 'question',
 'one',
 'answer',
 'stain',
 'cover',
 'heart',
 'tear',
 'apart',
 'like',
 'sleeping',
 'cancer',
 'believe',
 'men',
 'born',
 'killer',
 'believe',
 'world',
 'saved',
 'get',
 'start',
 'innocent',
 'child',
 'thorn',
 'heart',
 'kind',
 'world',
 'live',
 'love',
 'divided',
 'hate',
 'loosing',
 'control',
 'feeling',
 'must',
 'dreaming',
 'life',
 'away',
 'world',
 'cold',
 'sane',
 'shame',
 'moment',
 'time',
 'pass',
 'rewind',
 'blame',
 'start',
 'cure',
 'sickness',
 'heart',
 'believe',
 'men',
 'born',
 'killer',
 'believe',
 'world',
 'saved',
 'get',
 'start',
 'innocent',
 'child',
 'thorn',
 'heart',
 'kind',
 'world',
 'live',
 'love',
 'divided',
 'hate',
 'selling',
 'soul',
 'reason',
 'must',
 'dreaming',
 'life',
 'away',
 'world',
 'cold',
 'world',
 'cold',
 'sickness',
 'inside',
 'want',
 'escape',
 'feeling',
 'get',
 'find',
 'way',
 'many',
 'time',
 'must',
 'fall',
 'knee'

In [ ]:
df

,Genre,Lyrics
0,Rock,"[start, pain, followed, hate, fueled, endless,..."
1,Rock,"[freedom, alone, alone, patiently, waiting, ph..."
2,Rock,"[biting, hand, feed, lying, voice, inside, rea..."
3,Rock,"[say, know, imagine, wait, across, line, thoug..."
4,Rock,"[heart, beating, faster, control, feeling, any..."
...,...,...
290165,Metal,"[used, blind, living, inside, dream, someone, ..."
290166,Metal,"[trouble, pain, make, darkness, cry, scream, m..."
290169,Country,"[well, seen, lately, never, call, ask, friend,..."
290172,Country,"[rodeo, rodeo, riding, several, year, old, che..."


# Single Layer LSTM
For 1st set of results use:


    a) batch_size = 4
    b) max_sequence_length = 50
    c) embedding_dim = 50
    d) max_words = 10000
    e) lstm_units = 32

In [ ]:
batch_size = 4
max_sequence_length = 50
embedding_dim = 50
max_words = 10000
lstm_units = 32

In [ ]:
X = df['Lyrics'].values
y = df['Genre'].values

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = pad_sequences(X, maxlen=max_sequence_length)

In [ ]:
# Encode the genre labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Build single layer LSTM model

model_1 = Sequential()
model_1.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=50))
model_1.add(LSTM(lstm_units))
model_1.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            500000    
                                                                 
 lstm (LSTM)                 (None, 32)                10624     
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 510789 (1.95 MB)
Trainable params: 510789 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_1.fit(X_train, y_train, epochs=5, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/5
27179/27179 [==============================] - 358s 13ms/step - loss: 0.6346 - accuracy: 0.7868 - val_loss: 0.5775 - val_accuracy: 0.8073
Epoch 2/5
27179/27179 [==============================] - 357s 13ms/step - loss: 0.5300 - accuracy: 0.8246 - val_loss: 0.5625 - val_accuracy: 0.8143
Epoch 3/5
27179/27179 [==============================] - 356s 13ms/step - loss: 0.4794 - accuracy: 0.8419 - val_loss: 0.5640 - val_accuracy: 0.8145
Epoch 4/5
27179/27179 [==============================] - 359s 13ms/step - loss: 0.4355 - accuracy: 0.8562 - val_loss: 0.5783 - val_accuracy: 0.8142
Epoch 5/5
27179/27179 [==============================] - 358s 13ms/step - loss: 0.3937 - accuracy: 0.8707 - val_loss: 0.6168 - val_accuracy: 0.8047


In [ ]:
model_1.save('/content/drive/MyDrive/NLP Assignments/Models/1_model_1.keras')

In [ ]:
y_pred = model_1.predict(X_test)

y_pred = np.argmax(y_pred, axis=1)

from sklearn.metrics import classification_report

class_report_1 = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

850/850 [==============================] - 4s 4ms/step


<br>
<br>
<br>
<br>

# 2-Layer LSTM

For 2nd set of results use:

> Indented block
  a) batch_size = 8
  b) max_sequence_length = 30
  c) embedding_dim = 30
  d) max_words = 25000
  e) lstm_units = 32
  

In [ ]:
batch_size = 8
max_sequence_length = 30
embedding_dim = 30
max_words = 25000
lstm_units = 32

In [ ]:
X = df['Lyrics'].values
y = df['Genre'].values

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = pad_sequences(X, maxlen=max_sequence_length)

In [ ]:
# Encode the genre labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Build single layer LSTM model

model_2 = Sequential()
model_2.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=30))
model_2.add(LSTM(lstm_units))
model_2.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_2.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 30)            750000    
                                                                 
 lstm_1 (LSTM)               (None, 32)                8064      
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 758229 (2.89 MB)
Trainable params: 758229 (2.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_2.fit(X_train, y_train, epochs=5, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/5
13590/13590 [==============================] - 193s 14ms/step - loss: 0.6563 - accuracy: 0.7805 - val_loss: 0.6014 - val_accuracy: 0.7970
Epoch 2/5
13590/13590 [==============================] - 189s 14ms/step - loss: 0.5472 - accuracy: 0.8187 - val_loss: 0.5874 - val_accuracy: 0.8053
Epoch 3/5
13590/13590 [==============================] - 188s 14ms/step - loss: 0.4886 - accuracy: 0.8397 - val_loss: 0.5996 - val_accuracy: 0.8021
Epoch 4/5
13590/13590 [==============================] - 189s 14ms/step - loss: 0.4371 - accuracy: 0.8570 - val_loss: 0.6128 - val_accuracy: 0.7991
Epoch 5/5
13590/13590 [==============================] - 190s 14ms/step - loss: 0.3888 - accuracy: 0.8735 - val_loss: 0.6490 - val_accuracy: 0.7996


In [ ]:
model_2.save('/content/drive/MyDrive/NLP Assignments/Models/1_model_2.keras')

In [ ]:
y_pred = model_2.predict(X_test)

y_pred = np.argmax(y_pred, axis=1)

from sklearn.metrics import classification_report

class_report_2 = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

850/850 [==============================] - 3s 4ms/step


# Classification Reports
----
### Classification Report: Model 1

In [ ]:
print('Model 1')
print(class_report_1)
print('\n\nModel 2')
print(class_report_2)

Model 1
              precision    recall  f1-score   support

     Country       0.00      0.00      0.00       378
     Hip-Hop       0.64      0.41      0.50       447
        Jazz       0.74      0.56      0.64      2663
       Metal       0.59      0.55      0.57      3595
        Rock       0.85      0.91      0.88     20096

    accuracy                           0.80     27179
   macro avg       0.56      0.48      0.52     27179
weighted avg       0.79      0.80      0.79     27179



Model 2
              precision    recall  f1-score   support

     Country       0.00      0.00      0.00       378
     Hip-Hop       0.56      0.32      0.41       447
        Jazz       0.73      0.55      0.63      2663
       Metal       0.63      0.39      0.48      3595
        Rock       0.83      0.93      0.88     20096

    accuracy                           0.80     27179
   macro avg       0.55      0.44      0.48     27179
weighted avg       0.77      0.80      0.78     27179

